In [ ]:
# TODO: REMOVE ME!
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
import os
os.environ["MPLCONFIGDIR"] = "/tmp"

In [ ]:
import cv2
import itertools
import numpy as np
import os
import pandas as pd
import random
from IPython.display import HTML, display
from plaster.run.calib.calib import RegPSF, approximate_psf
from plaster.run.job import JobResult
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im, sigproc_v2_movie_from_df, sigproc_v2_im_from_df
from plaster.run.plots.plots_sigproc import wizard_xy_df, wizard_scat_df, wizard_raw_images
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat, mean_non_dark_asr
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image import imops
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.ipynb_helpers.displays import hd, movie
from plaster.tools.log.log import error, debug
from plaster.tools.schema import check
from plaster.tools.utils import data
from plaster.tools.utils import utils
from plaster.tools.zap import zap
from plaster.tools.zplots import zplots
from plaster.tools.ipynb_helpers import displays
from plaster.run.sigproc_v2 import bg
z = zplots.setup()

In [ ]:
# REMOVE ME
from plaster.run.base_result import disable_disk_memoize_from_notebook
disable_disk_memoize_from_notebook()

In [ ]:
job = JobResult.from_context(dev_override="/erisyon/internal/jobs_folder/abbe7_1t")
run = job.sigproc_v2

In [ ]:
hd("h1", f"All-cycles movie: field=median_quality")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    qdf = run.ims_import.qualities()
    qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)]#.sort_values(["quality"])
    mean_qdf = qdf.groupby("field_i").mean().reset_index().sort_values("quality")
    median_fl_i = int(mean_qdf.iloc[len(mean_qdf) // 2].field_i.astype(int))

    df = run.sigproc_v2.fields__n_peaks__peaks__radmat(fields=median_fl_i)
    dark = run.sigproc_v2.dark_estimate(ch_i=0, fields=median_fl_i)
    max_asr = run.sigproc_v2.asr_threshold()
    filt_df = df_filter(df, dark=dark, max_aspect_ratio=max_asr)
    sigproc_v2_movie_from_df(run, filt_df)

In [ ]:
hd("h1", f"All-fields movie: cycle=0")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    ims = np.zeros((run.sigproc_v2.n_fields, run.ims_import.dim, run.ims_import.dim))
    for fl_i in range(run.sigproc_v2.n_fields):
        im = run.sigproc_v2.aln_ims[fl_i, ch_i, 0]
        h, w = im.shape
        ims[fl_i, 0:h, 0:w] = im

    displays.movie(ims,
        _labels=[
            f"fl_i:{fl_i} ch_i:{ch_i} cy_i: 0"
            for fl_i in range(run.sigproc_v2.n_fields)
        ],
        _duration=200,
    )


In [ ]:
"""
Establishes “dark” mean and std.
Per cycle
Per field
Per channel
Foreground mean and std
Low-frequency power
A proxy for anomalies, oil bubbles, etc
High-frequency power
Should be a constant for a given camera
Changes to this statistic probably imply major camera issues 
"""

In [ ]:
n_cycles = run.ims_import.n_cycles

hd("h1", f"Pixel intensity percentiles=(0, 25, 50, 75, 99)<br/>over cycles, combining 5 random fields")
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    n_samples = 10000
    bg_samps = np.zeros((n_cycles, n_samples))
    fg_samps = np.zeros((n_cycles, n_samples))
    idx = np.arange(run.ims_import.dim**2)
    cy_labels = [f"cy {cy_i}" for cy_i in range(n_cycles)]
    for fl_i in utils.ispace(0, run.ims_import.n_fields, 5):
        for cy_i in range(n_cycles):
            im = run.ims_import.ims[fl_i, ch_i, cy_i]
            bg_im, fg_mask = bg.background_extract(im, approximate_psf(), dilate=0)
            _idx = data.subsample(idx[fg_mask.flatten()], n_samples)
            bg_samps[cy_i, :] = data.subsample(bg_im.flatten(), n_samples)
            fg_samps[cy_i, :] = im.flatten()[_idx]

    with z(_cols=3):
        z.distr(bg_samps, _vertical=500.0, _percentiles=(0, 25, 50, 75, 99), f_x_axis_label="Intensity", f_y_axis_label="Cycle", f_title="background pixels only")
        z.distr(fg_samps, _vertical=500.0, _percentiles=(0, 25, 50, 75, 99), f_x_axis_label="Intensity", f_y_axis_label="Cycle", f_title="foreground pixels only")
    
        bg_medians = np.nanmedian(bg_samps, axis=1)
        fg_medians = np.nanmedian(fg_samps, axis=1)
        z.scat(
            x=fg_medians,
            y=bg_medians,
            f_x_axis_label="FG Med. Inentsity", f_y_axis_label="BG Med. Inentsity",
            _label=cy_labels
        )
    

In [ ]:
# hd("h1", f"Calibration")
# if "calibration" not in run.sigproc_v2.params:
#     print("No calibration was provded to this run")
# else:
#     for ch_i in range(run.ims_import.n_channels):
#         hd("h2", f"Channel {ch_i}")

#         # Regional PSF
#         reg_psf = run.sigproc_v2.params.calibration.reg_psf(ch_i=ch_i)
#         check.t(reg_psf, RegPSF)
#         psf_ims = reg_psf.render()
#         plot_psfs(psf_ims, scale=3.0, f_title=f"ch_i={ch_i}", _noaxes=True, _notools=True)

#         # Regional illumination balance
#         illum = run.sigproc_v2.params.calibration.reg_illum().interp(ch_i)
#         z.im(1.0 / illum, f_title="Illumination map")


In [ ]:
# hd("h1", f"Field Quality")
# for ch_i in range(run.ims_import.n_channels):
#     hd("h2", f"Channel {ch_i}")
#     qdf = run.ims_import.qualities()
#     qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)].sort_values(["quality"])
#     z.hist(qdf.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution channel {ch_i}")

#     row_iz = utils.ispace(0, len(qdf), 3)

#     # COMBINE all images for common percentile calculations
#     ims = np.concatenate([
#         run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i].flatten()
#         for row in qdf.iloc[row_iz].itertuples()
#     ])
#     bot, top = np.percentile(ims, (50, 99))

#     # SHOW example of worst, median, and best all using the same cspan
#     hd("h3", f"Examples of frames by quality")
#     with z(_cols=3, _cspan=(bot, top)):
#         names = ("worst", "median", "best")
#         for name, row in zip(names, qdf.iloc[row_iz].itertuples()):
#             z.im(run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i], f_title=f"Channel: {ch_i} {name}")

In [ ]:
# hd("h1", f"Field Alignment")
# field_df = run.sigproc_v2.fields().copy()
# field_df["alignment"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
# alignment = field_df.groupby("field_i").alignment.max().values
# z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="n_pixels", f_title="Max. alignment dist.")